In [1]:
import findspark
import subprocess
from pyspark.sql import SparkSession


ip_addresses = subprocess.check_output(['hostname', '-I']).decode().strip()
print(ip_addresses)
findspark.init()


spark = SparkSession.builder \
    .appName("PySparkJupyter") \
    .config("spark.ui.enabled", "false") \
    .config("spark.driver.bindAddress", ip_addresses) \
    .config("spark.driver.host", ip_addresses) \
    .getOrCreate()

172.22.6.46


your 131072x1 screen size is bogus. expect trouble
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/07/30 07:52:28 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
DF_Extract = spark.read \
    .format("csv") \
    .option("inferschema","true") \
    .option("header","true") \
    .load("/home/airflowsup/ETL_PRJ_2/orchestrated/airflow-extract-data_2.csv")

# DF_Extract.show()

In [3]:
from pyspark.sql import functions as F
from pyspark.sql import Window as W

SectorWindow = W.partitionBy(["Date added","GICS Sector"])
Output_Columns = ["Date added","GICS Sector","Aggregated_Count"]

DF_Transform = DF_Extract.drop_duplicates()
DF_Transform = DF_Transform.withColumn("Aggregated_Count", F.count(F.col("Symbol")).over(SectorWindow)) \
                          .select([F.col(f"{i}") for i in Output_Columns]).drop_duplicates()     

DF_Transform.count()

435

In [ ]:
!jupyter nbconvert --to script /home/airflowsup/ETL_PRJ_2/orchestrated/ETL_PRJ_2_PysparkScript.ipynb